In [322]:
# general imports
import matplotlib.pyplot as plt
# magic word for producing visualizations in notebook
%matplotlib inline
import string
import time
import numpy as np
import random
from scipy.linalg import expm

# AWS imports: Import Braket SDK modules
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice

In [323]:
# function to initialize the input qubits and output qubit

def init_states(n_qubits): 
    
    # define the output qubit
    output_qubit = n_qubits 
    
    # create circuit and initialize states
    circuit = Circuit().h(range(n_qubits)).x(output_qubit).h(output_qubit)
    
    return circuit

In [324]:
# example circuit initializition for 3 qubits 
n_qubits = 3
init = init_states(n_qubits)
print(init)

T  : |0|1|
          
q0 : -H---
          
q1 : -H---
          
q2 : -H---
          
q3 : -X-H-

T  : |0|1|


In [325]:
# function to create a constant oracle 

def constant_oracle(n_qubits):
    
    # define the output qubit 
    output_qubit = n_qubits 
    
    # create circuit 
    circuit = Circuit()
    
    # set output to either 0 or 1 
    rand_output = np.random.randint(0, 2)
    
    # if output qubit is 0, apply i gate to not change value 
    if rand_output == 0:
        circuit.i(output_qubit)
    # if output is 1, apply x gate to change value to 0 
    if rand_output == 1: 
        circuit.x(output_qubit)
    
    return circuit

In [326]:
# example circuit for constant oracle 
n_qubits = 5
const = constant_oracle(n_qubits)
print(const) 

T  : |0|
        
q5 : -I-

T  : |0|


In [327]:
# function to create a balanced oracle 

def balanced_oracle(n_qubits): 
    
    # define the output qubit 
    output_qubit = n_qubits 
    
    # create circuit 
    circuit = Circuit()
    
    # generate a random array of 0s and 1s to figure out where to place x gates
    random_num = np.random.randint(2, size=n_qubits)
    
    # place x gates 
    for qubit in range(len(random_num)):
        if random_num[qubit] == 1:
            circuit.x(qubit)        
    
    # place cnot gates
    for qubit in range(n_qubits):
        circuit.cnot(control=qubit, target=output_qubit)
    
    # place x gates    
    for qubit in range(len(random_num)): 
        if random_num[qubit] == 1:
            circuit.x(qubit)
        
    return circuit

In [328]:
# example circuit for balanced oracle
n_qubits = 10
bal = balanced_oracle(n_qubits)
print(bal)

T   : |0|1|2|3|4|5|6|7|8|9|10|11|
                                 
q0  : -X-C-X---------------------
         |                       
q1  : -X-|-C-X-------------------
         | |                     
q2  : ---|-|-C-------------------
         | | |                   
q3  : ---|-|-|-C-----------------
         | | | |                 
q4  : -X-|-|-|-|-C-X-------------
         | | | | |               
q5  : -X-|-|-|-|-|-C-X-----------
         | | | | | |             
q6  : ---|-|-|-|-|-|-C-----------
         | | | | | | |           
q7  : ---|-|-|-|-|-|-|-C---------
         | | | | | | | |         
q8  : ---|-|-|-|-|-|-|-|-C-------
         | | | | | | | | |       
q9  : -X-|-|-|-|-|-|-|-|-|-C--X--
         | | | | | | | | | |     
q10 : ---X-X-X-X-X-X-X-X-X-X-----

T   : |0|1|2|3|4|5|6|7|8|9|10|11|


In [329]:
def random_oracle(n_qubits): 
    
    # create circuit 
    circuit = Circuit()
    
    # define the output qubit 
    output_qubit = n_qubits 
    
    # create a random array of 0s and 1s to determine where to place the gates
    random_array = np.random.randint(2, size=n_qubits)
    
    # define the single gate set 
    single_gate_set = "x"
    
    # define the multiple qubit gate set
    multiple_gate_set = ["cnot", "ccnot"]
    
    # create a list combining single and multiple qubit gates to randomly choose from
    choice_gate_set = ["single_gate_set", "multiple_gate_set"]
    
    # random oracle circuit generator
    
    circuit = Circuit()
    
    # implement all the gate options 
    for qubit in range(len(random_array)):
        
        # randomly choose to apply a single qubit or multiple qubit gate 
        random_gate_set = np.random.choice(choice_gate_set, p=[0.30, 0.70])
        
        # if single qubit gate then implement x gate accordingly 
        if random_gate_set == "single_gate_set": 
            if random_array[qubit] == 1:
                circuit.x(qubit)
        
        # if multiple qubit gate then implement cnot and ccnot gates 
        if random_gate_set == "multiple_gate_set": 
            
            # randomly choose to implement a cnot or ccnot gate
            random_gate_m = np.random.choice(multiple_gate_set)
            if random_gate_m == "cnot":
                if random_array[qubit] == 0:
                    #randomly choose where the target qubit is
                    targetf = np.random.randint(n_qubits+1)
                    if qubit!= targetf:
                        circuit.cnot(control=qubit, target=targetf)
                    del targetf
                else: 
                    #randomly choose where the target qubit is
                    targetf = np.random.randint(n_qubits+1)
                    circuit.x(qubit)
                    if qubit!= targetf:
                        circuit.cnot(control=qubit, target=targetf)
                    circuit.x(qubit) 
                    del targetf
            if random_gate_m == "ccnot":
                #randomly choose where the first and second controls are
                control1 = np.random.randint(n_qubits+1)
                control2 = np.random.randint(n_qubits+1)
                #randomly choose where the target qubit is
                targetf = np.random.randint(n_qubits+1)
                if control1 != control2 and control1 != targetf and control2 != targetf:
                    circuit.ccnot(control1, control2, targetf) 
                del control1
                del control2
                del targetf
    
    return circuit

In [330]:
n_qubits = 4 
random = random_oracle(n_qubits)
print(random)

T  : |0|
        
q1 : -X-
      | 
q3 : -C-
      | 
q4 : -C-

T  : |0|


In [349]:
# general function for taking an dj-algorithm 

def dj_algorithm(case, n_qubits):
    
    # create circuit 
    circuit = Circuit()
    
    # define the output qubit 
    output_qubit = n_qubits 
    
    # create circuit and initialize states
    circuit = Circuit().h(range(n_qubits)).x(output_qubit).h(output_qubit)
    
    # creating a circuit for constant oracle
    if case == "constant":
        
        # set output to either 0 or 1 
        rand_output = np.random.randint(0, 2)
    
        # if output qubit is 0, apply i gate to not change value 
        if rand_output == 0:
            circuit.i(output_qubit)
        # if output is 1, apply x gate to change value to 0 
        if rand_output == 1: 
            circuit.x(output_qubit)
    
    # create a circuit for balanced oracle
    if case == "balanced":
        
        # generate a random array of 0s and 1s to figure out where to place x gates
        random_num = np.random.randint(2, size=n_qubits)
    
        # place x gates 
        for qubit in range(len(random_num)):
            if random_num[qubit] == 1:
                circuit.x(qubit)
            
        # place cnot gates
        for qubit in range(n_qubits):
            circuit.cnot(control=qubit, target=output_qubit)
    
        # place x gates    
        for qubit in range(len(random_num)): 
            if random_num[qubit] == 1:
                circuit.x(qubit)
    
    # create a completely random oracle 
    if case == "random":
        
        # create a random array of 0s and 1s to determine where to place the gates
        random_array = np.random.randint(2, size=n_qubits)
    
        # define the single gate set 
        single_gate_set = "x"
    
        # define the multiple qubit gate set
        multiple_gate_set = ["cnot", "ccnot"]
    
        # create a list combining single and multiple qubit gates to randomly choose from
        choice_gate_set = ["single_gate_set", "multiple_gate_set"]
    
        # random oracle circuit generator
    
        circuit = Circuit()
    
        # implement all the gate options 
        for qubit in range(len(random_array)):
        
            # randomly choose to apply a single qubit or multiple qubit gate 
            random_gate_set = np.random.choice(choice_gate_set, p=[0.30, 0.70])
        
            # if single qubit gate then implement x gate accordingly 
            if random_gate_set == "single_gate_set": 
                if random_array[qubit] == 1:
                    circuit.x(qubit)
        
            # if multiple qubit gate then implement cnot and ccnot gates 
            if random_gate_set == "multiple_gate_set": 
            
                # randomly choose to implement a cnot or ccnot gate
                random_gate_m = np.random.choice(multiple_gate_set)
                if random_gate_m == "cnot":
                    if random_array[qubit] == 0:
                        #randomly choose where the target qubit is
                        targetf = np.random.randint(n_qubits+1)
                        if qubit!= targetf:
                            circuit.cnot(control=qubit, target=targetf)
                        del targetf
                    else: 
                        #randomly choose where the target qubit is
                        targetf = np.random.randint(n_qubits+1)
                        circuit.x(qubit)
                        if qubit!= targetf:
                            circuit.cnot(control=qubit, target=targetf)
                        circuit.x(qubit) 
                        del targetf
                if random_gate_m == "ccnot":
                    #randomly choose where the first and second controls are
                    control1 = np.random.randint(n_qubits+1)
                    control2 = np.random.randint(n_qubits+1)
                    #randomly choose where the target qubit is
                    targetf = np.random.randint(n_qubits+1)
                    if control1 != control2 and control1 != targetf and control2 != targetf:
                        circuit.ccnot(control1, control2, targetf) 
                    del control1
                    del control2
                    del targetf
    
    # place the h-gates again
    circuit.h(range(n_qubits))
    
    # measure the results
    for qubit in range(n_qubits):
        circuit.sample(observable=Observable.Z(), target=qubit)
    Shots = 10
    #Designate the device being used as the local simulator
    #Feel free to use another device
    device = LocalSimulator()
    #Submit the task
    my_task = device.run(circuit, shots=Shots)
    #Retrieve the result
    result = my_task.result()
    #print the measurement probabilities
    print('Measurement results:\n',result.measurement_probabilities) 
    
    return circuit
  

In [350]:
n_qubits = 4
dj = dj_algorithm("balanced", n_qubits)
print(dj)

Measurement results:
 {'11110': 0.7, '11111': 0.3}
T  : |0|1|2|3|4|5|6|Result Types|
                               
q0 : -H---C-H-------Sample(Z)----
          |                    
q1 : -H---|-C-H-----Sample(Z)----
          | |                  
q2 : -H-X-|-|-C-X-H-Sample(Z)----
          | | |                
q3 : -H---|-|-|-C-H-Sample(Z)----
          | | | |              
q4 : -X-H-X-X-X-X----------------

T  : |0|1|2|3|4|5|6|Result Types|


In [351]:
# run initialized states through circuit and add random_oracle 
def classical_checker(n_qubits):
    
    # define the output qubit 
    output_qubit = n_qubits 

    n = n_qubits
    
    matrix = []
    
    for i in range(0, 2**n):
        # convert integer to bitstring and into an integer
        bit_str = [int(x) for x in str(format(i, "b"))]
        # if len of bitstring less than n then pad with zeros
        if len(bit_str) < n:
            num_zero = n - len(bit_str)
            bit_str = np.pad(bit_str, (num_zero, 0), 'constant')
        # append bit_str to matrix 
        matrix.append(bit_str)
    
    # turn matix into an numpy array
    matrix_bitout = np.array(matrix)
    # create a random array of 0s and 1s to determine where to place the gates
    random_array = np.random.randint(2, size=n_qubits)
    
    # define the single gate set 
    single_gate_set = "x"
    
    # define the multiple qubit gate set
    multiple_gate_set = ["cnot", "ccnot"]
    
    # create a list combining single and multiple qubit gates to randomly choose from
    choice_gate_set = ["single_gate_set", "multiple_gate_set"]
    
    # random oracle circuit generator
    
    random_circuit = Circuit()
    
    # implement all the gate options 
    for qubit in range(len(random_array)):
        
        # randomly choose to apply a single qubit or multiple qubit gate 
        random_gate_set = np.random.choice(choice_gate_set, p=[0.30, 0.70])
        
        # if single qubit gate then implement x gate accordingly 
        if random_gate_set == "single_gate_set": 
            if random_array[qubit] == 1:
                random_circuit.x(qubit)
        
        # if multiple qubit gate then implement cnot and ccnot gates 
        if random_gate_set == "multiple_gate_set": 
            
            # randomly choose to implement a cnot or ccnot gate
            random_gate_m = np.random.choice(multiple_gate_set)
            if random_gate_m == "cnot":
                if random_array[qubit] == 0:
                    #randomly choose where the target qubit is
                    targetf = np.random.randint(n_qubits+1)
                    if qubit!= targetf:
                        random_circuit.cnot(control=qubit, target=targetf)
                    del targetf
                else: 
                    #randomly choose where the target qubit is
                    targetf = np.random.randint(n_qubits+1)
                    random_circuit.x(qubit)
                    if qubit!= targetf:
                        random_circuit.cnot(control=qubit, target=targetf)
                    random_circuit.x(qubit) 
                    del targetf
            if random_gate_m == "ccnot":
                #randomly choose where the first and second controls are
                control1 = np.random.randint(n_qubits+1)
                control2 = np.random.randint(n_qubits+1)
                #randomly choose where the target qubit is
                targetf = np.random.randint(n_qubits+1)
                if control1 != control2 and control1 != targetf and control2 != targetf:
                    random_circuit.ccnot(control1, control2, targetf) 
                del control1
                del control2
                del targetf
    
    
    measurement_results_classical = []
    
    
    for row in matrix_bitout:
        circuit = Circuit()
        #retrieve the bit string that initializes the qubits for this iteration
        qubit_info = row.tolist()
        print(qubit_info) 
        # creating initial states of our classical circuit
        for qubit in range(len(qubit_info)):
            if qubit_info[qubit] == 1:
                circuit.x(qubit)
            else: 
                circuit.i(qubit)
        # adding random circuit to our initial states 
        circuit.add_circuit(random_circuit, range(n_qubits))
        circuit.sample(observable=Observable.Z(), target=output_qubit)
        print(circuit)
        
        Shots = 1
        #Designate the device being used as the local simulator
        #Feel free to use another device
        device = LocalSimulator()
        #Submit the task
        my_task = device.run(circuit, shots=Shots)
        
        #Retrieve the result
        result = my_task.result()
        #print the measurement probabilities
        print('Measurement results:\n',result.measurement_probabilities)
        del circuit
        
        # store results in a measurement variable 
        measurement_classical = result.measurement_probabilities
        for k, v in measurement_classical.items(): 
            int(k)
        measurement_list = (list(k))
        measurement_results_classical.append(measurement_list)
        # classical checker circuits 
        classical_results = np.array(measurement_results_classical)
        final_rc = []
        for row in classical_results: 
            result = int(row[-1])
            final_rc.append(result)
    
        total_rc = len(final_rc)
        count_0 = 0
        count_1 = 0
    
        for val in final_rc:
            if val == 0: 
                count_0 += 1
            if val == 1:
                count_1 += 1
    
    if total_rc == count_0: 
        print("This is a constant function")
    
    elif count_0 == total_rc/2 and count_1 == total_rc/2: 
            print("This is a balanced function")
    else:
        print("This is neither a constant nor balanced function")
         
     
    print("These are the final outputs from the random circuits", final_rc)
    return random_circuit

In [352]:
n_qubits = 3
run_circuit = classical_checker(n_qubits)

[0, 0, 0]
T  : |0|1|Result Types|
                     
q0 : -I-X--------------
                     
q1 : -I----------------
                     
q2 : -I----------------
                     
q3 : -----Sample(Z)----

T  : |0|1|Result Types|
Measurement results:
 {'1000': 1.0}
[0, 0, 1]
T  : |0|1|Result Types|
                     
q0 : -I-X--------------
                     
q1 : -I----------------
                     
q2 : -X----------------
                     
q3 : -----Sample(Z)----

T  : |0|1|Result Types|
Measurement results:
 {'1010': 1.0}
[0, 1, 0]
T  : |0|1|Result Types|
                     
q0 : -I-X--------------
                     
q1 : -X----------------
                     
q2 : -I----------------
                     
q3 : -----Sample(Z)----

T  : |0|1|Result Types|
Measurement results:
 {'1100': 1.0}
[0, 1, 1]
T  : |0|1|Result Types|
                     
q0 : -I-X--------------
                     
q1 : -X----------------
                     
q2 : -X--------

In [353]:
def random_oracle(n_qubits,random_circuit):
    # create circuit 
    circuit2 = Circuit()
    
    # define the output qubit 
    output_qubit = n_qubits 
    
    # create circuit and initialize states
    circuit2 = Circuit().h(range(n_qubits)).x(output_qubit).h(output_qubit)
    #add the random oracle to this circuit
    circuit2.add_circuit(random_circuit, range(n_qubits))
    # place the h-gates again
    circuit2.h(range(n_qubits))
    
    measurement_results_quantum = []
    
    # measure the results
    for qubit in range(n_qubits):
        circuit2.sample(observable=Observable.Z(), target=qubit)
    print(circuit2)
    Shots = 100
    #Designate the device being used as the local simulator
    #Feel free to use another device
    device = LocalSimulator()
    #Submit the task
    my_task2 = device.run(circuit2, shots=Shots)
    #Retrieve the result
    result2 = my_task2.result()
    measurement_quantum = result2.measurement_probabilities
    #print the measurement probabilities
    print('Measurement results:\n',result2.measurement_probabilities)
    
    measurement_keys = []
    for k, v in measurement_quantum.items():
        measurement_keys.append(list(k))
    quantum_results = np.array(measurement_keys)
    final_rq = []
    all_measurements = []
    for row in quantum_results:
        func = []
        for val in row[:-1]:
            func.append(int(val))
        all_measurements.append(func)
    
    num_outputs = len(all_measurements)
    
    type_func = 0
    
    for row in all_measurements:
        if sum(row) == 0: 
            type_func += 0
        else: 
            type_func += 1

    if type_func == 0: 
        print("This is a constant function")
    elif type_func == num_outputs: 
        print("This is a balanced function")
    else: 
        print("This is neither a balanced or constant function")

In [354]:
n_qubits = 3 
oracle = random_oracle(n_qubits,run_circuit)

T  : |0|1|2|Result Types|
                       
q0 : -H-X-H-Sample(Z)----
                       
q1 : -H-H---Sample(Z)----
                       
q2 : -H-H---Sample(Z)----
                       
q3 : -X-H----------------

T  : |0|1|2|Result Types|
Measurement results:
 {'0001': 0.48, '0000': 0.52}
This is a constant function
